In [28]:
# Imports and versions
import os, sys, platform, subprocess
from pathlib import Path
import pandas as pd
import plotly
import plotly.express as px

print('Python:', sys.version.splitlines()[0])
print('Pandas:', pd.__version__)
print('Plotly:', plotly.__version__)

Python: 3.12.11 (main, Jul  1 2025, 05:29:09) [GCC 12.2.0]
Pandas: 2.3.3
Plotly: 6.5.0


In [29]:
# Paths to key CSVs (using absolute paths from repo root)
from pathlib import Path
REPO_ROOT = Path('/workspaces/WjecDocumentScraper')
DATA_DOCS = REPO_ROOT / 'document_stats.csv'
DATA_FILES = REPO_ROOT / 'notebooks/cleansed_data/document_stats-files.csv'
DATA_LLM_PROOFREADER = REPO_ROOT / 'notebooks/cleansed_data/llm_proofreader_cleansed_data.csv'
DATA_CATEGORISED = REPO_ROOT / 'notebooks/cleansed_data/language_check_categorised.csv'

# Load (these are small enough to load into memory in this repo)
docs = pd.read_csv(DATA_DOCS)
files = pd.read_csv(DATA_FILES)
llm_proofreader = pd.read_csv(DATA_LLM_PROOFREADER)

files.shape, llm_proofreader.shape, docs.shape

((239, 3), (15673, 11), (25, 4))

In [30]:
# Reproducibility: git commit and environment
try:
    sha = subprocess.check_output(['git','rev-parse','--short','HEAD']).decode().strip()
except Exception:
    sha = '<not available>'
print('Git commit:', sha)
print('Platform:', platform.platform())

Git commit: f1bda30
Platform: Linux-6.8.0-1030-azure-x86_64-with-glibc2.36
